In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import StructuredTool

llm = ChatOpenAI(temperature=0.1)
def plus(a,b):
    return a+b

agent = initialize_agent(
    llm=llm, 
    verbose=True,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    tools=[
    StructuredTool.from_function(
        func=plus,
        name="Sum Calculator",
        description="Use this to perform sums of two numbers. This tool take two arguments, both should be numbers.",
    )
])
prompt = "Cost of $325.34 + $12.32 + $999.23 + $23843.34"

agent.invoke(prompt)

ImportError: cannot import name 'create_structured_chat_agent' from 'langchain.agents' (c:\Users\gina0\Desktop\Pythoncode\fullstack-gpt\env\Lib\site-packages\langchain\agents\__init__.py)